# Auslesen der Management-Transaktionen bei SIX

 Diese Anwendung basiert logisch auf "Selenium auf SIX Management Transaktionen". Ziel ist jedoch, die einzelne Selenium-Abfrage in eine funktion zu Packen, die dann wiederum in Schleifen aufgerufen werden kann.
 
 Die Idee ist, die Abfrage in Selenium und die Auswertung in separate Funktionen auszulagern und diese am ende mit einem kurzen Code automatisiert zu starten. 
 
 Dabei wird erst die Funktion selenium() aufgerufen. Das Ergebnis geht dann an die Funktion suppe(), welche ihrerseits wiederum in einer Schleife die Funktion meldungen() einsetzt.
 
 Aktuelles Problem: Obwohl Selenium die richtigen Daten setzt und die Abfragen auch die richtigen Daten liefert, erscheinen am Ende Werte aus einem anderen Monat im Endergebnis. Da dies bei getrennten Einzelausführungen nicht der Fall ist, muss der Fehler am Code liegen und nicht an der Website.

## Allgemeine Vorbereitung

In [1]:
#Importe
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import os
import pandas as PD
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


 # Selenium-Abfrage als Funktion
 
 Die Funktion soll nach Eingabe eines Start- und Enddatums einen HTML-Code des Seite ausgeben.

In [7]:
#Ab der SIX-Seite für ein bestimmtes Datum alle meldungen auslesen und als HTML speichern.
#Ausgabe ist website als html. 

def selenium(startdatum, enddatum):
    #definition webdriver (ohne Pfadangabe, da Pfad gespeichert)
    driver = webdriver.Chrome()
    
    #definition Website
    driver.get("https://www.six-exchange-regulation.com/de/home/publications/management-transactions.html")
    #driver.get("https://www.six-exchange-regulation.com/de/home/publications/management-transactions.html#notificationId=T1I2G00075")
    
    time.sleep(1)           
               
    #datum setzen
    datum_von = driver.find_element_by_id('FromTransactionDate')
    datum_von.click()
    datum_von.clear()
    datum_von.send_keys(startdatum)          

    datum_bis = driver.find_element_by_name('ToTransactionDate')
    datum_bis.click()
    datum_bis.clear()
    datum_bis.send_keys(enddatum)
               
    time.sleep(1)           
    
    #Alle Firmen wählen
    #select = Select(driver.find_element_by_name("NotificationSubmitterId"))
    #select.select_by_value("") #Für alle Aktien
    
    #Einfach nur Abschicken
    datum_von = driver.find_element_by_id('FromTransactionDate')
    datum_von.send_keys(Keys.ENTER)
    
    #time.sleep(5)   
    
    #HTML-Inhalt erfassen und übergeben
    html = driver.page_source

    return html 

In [8]:
#test
liste=[]

start = "01012018"
ende = "20012018"

selenium(start, ende)

# Suppe als Funktion

Diese Funktion soll den über Selenium() generierten Code auswerten und als Liste ausgeben. 

## Aus der Website die einzelnen Zeilen auslesen

In [4]:
def suppe(site):
    liste = []
    website = str(site)
    soup = BeautifulSoup(website, 'lxml')
    ergebniszeilen_gerade = soup.find_all("tr", class_="itf-mark-row-template even")
    ergebniszeilen_ungerade = soup.find_all("tr", class_="itf-mark-row-template odd")
    allezeilen = ergebniszeilen_gerade + ergebniszeilen_ungerade
    
   
    for meldung in ergebniszeilen_ungerade:
        meldung = str(meldung)
        meldungen(meldung)

    for meldung in ergebniszeilen_gerade:
        meldung = str(meldung)
        meldungen(meldung)

    return liste

In [9]:
#test
#liste = []
suppe(site)
liste

In [151]:
#die liste löschen
liste = []

## Aus den extrahierten Meldungen die Daten auslesen

Diese Funktion ist eine Unterfunktion von suppe(), die von suppe() in einer Schleife aufgerufen wird.

In [5]:
#Aus einer Datenzeile die Meldungen extrahieren

def meldungen(zeile):

    soup_zeile=BeautifulSoup(zeile, 'lxml')

    datum = soup_zeile.select('[data-itf-inject]')[0].text
    unternehmen = soup_zeile.select('[data-itf-inject]')[1].text
    art = soup_zeile.select('[data-itf-inject]')[2].text
    #anzahl = float(soup_zeile.select('[data-itf-inject]')[3].text.replace("'",""))
    anzahl = soup_zeile.select('[data-itf-inject]')[3].text.replace("'","")
    wertpapier = soup_zeile.select('[data-itf-inject]')[4].text
    #wert = float(soup_zeile.select('[data-itf-inject]')[5].text.replace("'",""))
    wert = soup_zeile.select('[data-itf-inject]')[5].text.replace("'","")

    #die umwandlung in float scheint probleme zu machen, daher wurde sie mal noch ausgelassen

    #Diese einzelne Zeile in ein Minidict schreiben
    #Minidict erstellen

    dict_meldung = {"datum" : datum, "unternehmen":unternehmen, "art":art, "anzahl":anzahl, "wertpapier":wertpapier, "wert":wert}
    liste.append(dict_meldung)

# Ergebnis anzeigen

Das Ergebnis in einem Panda darstellen und exportieren

In [10]:
#Panda erstellen

datensatz = pd.DataFrame(liste)
datensatz
#datensatz.plot()

# Alles hinter einander ausführen

In [11]:
start = "01012018"
ende = "10012018"

liste = []
site = selenium(start, ende)
suppe(site) 
datensatz = pd.DataFrame(liste)
datensatz